In [1]:
import torch

In [91]:
class GAN(torch.nn.Module):
    
    def __init__(self, convtrans_channel=[6,3,3],z_dim=8,channel_input=8):
        
        super(type(self), self).__init__()
        self.z_dim=z_dim
        self.batch=32
        self.channel=channel_input
        
        ##capas generadoras
        self.generative_layers = torch.nn.Sequential(
                                  torch.nn.ConvTranspose2d(self.channel,convtrans_channel[0],4,stride=2,padding=2
                                                          ),
                                  torch.nn.ReLU(),
                                  torch.nn.ConvTranspose2d(convtrans_channel[0],convtrans_channel[1],5,padding=4),
                                  torch.nn.ReLU(),
                                  torch.nn.ConvTranspose2d(convtrans_channel[1],convtrans_channel[2],3,stride=2),
                                  
            
                                )
        
        
        self.activation=torch.nn.ReLU()
        
    
    def generador(self, z):
        x_falso=self.generative_layers(z)
        return x_falso
        
    def discriminador(self,z):
        
        return z
    
    def sample(self):
        z=torch.randn(self.batch,self.channel,self.z_dim,self.z_dim)
        x_falso=self.generador(z)
        
        return z
    
    def forward(self):
        z=self.sample()
        x_falso=self.generador(z)
        print(x_falso.shape)
    
###preguntas: 1- sampleo, está bien un cubo desde una distribucion normal estandar 0,1?
###pregunta 2: dimension de HitS, 3 canales con dimension 21,21? o 1 canal con dimension 21,21*21 ?? como podria afectar una u otra forma?

In [92]:
model=GAN()

In [93]:
model.forward()

torch.Size([32, 3, 21, 21])
